In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

from statsmodels.tsa.holtwinters import SimpleExpSmoothing # SES
from statsmodels.tsa.holtwinters import Holt # Holts Exponential Smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing 

In [ ]:
df = pd.read_csv('AAPL.csv')

In [ ]:
Train = df.head(1760)
Test = df.tail(251)

In [ ]:
Train

In [ ]:
Test

In [ ]:
plt.figure(figsize=(24,7))
df.Close.plot(label="org")
for i in range(2,30,6):
    df["Close"].rolling(i).mean().plot(label=str(i))
plt.legend(loc='best')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decompose_ts_add = seasonal_decompose(df.Close,period= 12)
decompose_ts_add.plot()
plt.show()

In [ ]:
import statsmodels.graphics.tsaplots as tsa_plots
tsa_plots.plot_acf(df.Close,lags=12)
tsa_plots.plot_pacf(df.Close,lags=12)
plt.show()


In [ ]:
def MAPE(pred,org):
    temp = np.abs((pred-org)/org)*100
    return np.mean(temp)

Simple Exponential Method

In [ ]:
ses_model = SimpleExpSmoothing(Train["Close"]).fit(smoothing_level=0.2)
pred_ses = ses_model.predict(start = Test.index[0],end = Test.index[-1])
MAPE(pred_ses,Test.Close) 

Holt method

In [ ]:
# Holt method 
hw_model = Holt(Train["Close"]).fit(smoothing_level=0.8, smoothing_trend=0.2)
pred_hw = hw_model.predict(start = Test.index[0],end = Test.index[-1])
MAPE(pred_hw,Test.Close) 

Holts winter exponential smoothing with additive seasonality and additive trend

In [ ]:
hwe_model_add_add = ExponentialSmoothing(Train["Close"],seasonal="add",trend="add",seasonal_periods=251).fit() #add the trend to the model
pred_hwe_add_add = hwe_model_add_add.predict(start = Test.index[0],end = Test.index[-1])
MAPE(pred_hwe_add_add,Test.Close) 

Holts winter exponential smoothing with multiplicative seasonality and additive trend

In [ ]:
hwe_model_mul_add = ExponentialSmoothing(Train["Close"],seasonal="mul",trend="add",seasonal_periods=251)

In [ ]:
model_fit = hwe_model_mul_add.fit()

In [ ]:
pred_hwe_mul_add = model_fit.predict(start = Test.index[0],end = Test.index[-1])
MAPE(pred_hwe_mul_add,Test.Close)

In [ ]:
model_fit = ExponentialSmoothing(df.Close,seasonal="add",trend="add",seasonal_periods=251).fit()

In [ ]:
# make predictions on the test data
predictions = model_fit.forecast(30)

In [ ]:
predictions

Final Model by combining train and test

In [ ]:
hw_model = ExponentialSmoothing(df.Close,seasonal="add",trend="add",seasonal_periods=251).fit()

In [ ]:
hw_model.forecast(30)

In [ ]:
y_pred = hw_model.predict(start=len(Train), end=len(Train)+len(Test)-1)

from sklearn.metrics import mean_squared_error
rmse2 = mean_squared_error(Test['Close'].astype(float), y_pred, squared=False)
rmse2

In [ ]:
y_pred

In [ ]:
len(y_pred)

In [ ]:
comp = pd.DataFrame()
comp['test'] = Test["Close"]
comp['prediction'] = y_pred
comp

In [ ]:
plt.plot(Test["Close"], color = 'red', label = 'Real  Stock Price')
plt.plot(y_pred, color = 'blue', label = 'Predicted  Stock Price')
plt.title(' Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel(' Stock Price')
plt.legend()
plt.show()

In [ ]:
import pickle

In [ ]:
with open('apple_stock_model.pkl', 'wb') as f:
    pickle.dump(model_fit, f)

In [4]:
import streamlit as st
import pickle
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing 
import plotly.express as px

st.title('Apple Stock Price Forecasting')

# load the saved model
with open('apple_stock_model.pkl', 'rb') as f:
    model_fit = pickle.load(f)

# load the test dataset
df_test = pd.read_csv('AAPL.csv', parse_dates=['Date'], index_col='Date')

# Split the data into train and test sets
Train = df_test.head(1760)
Test = df_test.tail(251)

# Add a widget to trigger the forecasting and chart generation
if st.button('Generate Forecast'):
    # Fit the Prophet model on the training data
    hwe_model_mul_add = ExponentialSmoothing(Train["Close"],seasonal="mul",trend="add",seasonal_periods=251).fit() 

    # Forecast for the next N days
    forecast_period = st.number_input("Enter the number of days to forecast", min_value=1, max_value=None, value=30)
    forecast_dates = pd.date_range(start=df_test.index[-1], periods=forecast_period, freq='D')
    forecast_data = pd.DataFrame(index=forecast_dates, columns=df_test.columns)
    forecast_data['Open'] = df_test['Open'][-1]
    forecast_data['High'] = df_test['High'][-1]
    forecast_data['Low'] = df_test['Low'][-1]
    forecast_data['Close'] = hwe_model_mul_add.predict(pd.DataFrame(forecast_dates, columns=['ds'])).yhat.values

    # Make predictions for the test data
    Test = Test.sort_index()
    pred_hwe_mul_add = hwe_model_mul_add.predict(pd.DataFrame(Test.index, columns=['ds'])).yhat.values

    # Add a checkbox for showing the forecast
    show_forecast = st.checkbox('Show forecast')

    # Plot the actual and predicted values for the test data and forecasted values
    fig = px.line(pd.concat([Test.Close, pd.Series(pred_hwe_mul_add, index=Test.index)], axis=1).reset_index().melt(id_vars='Date', value_vars=['Close', 0]), x='Date', y='value', color='variable')
    if show_forecast:
        fig.add_trace(px.line(forecast_data.reset_index().melt(id_vars='Date', value_vars=['Close']), x='Date', y='value').data[0])
    fig.update_layout(title='Apple Stock Prices', xaxis_title='Date', yaxis_title='Price', legend_title='')
    st.plotly_chart(fig)
